In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import pandas as pd
import regex as re
import string
import numpy as np
import evaluate

The following script is based on https://huggingface.co/docs/transformers/tasks/sequence_classification.

In [2]:
# load the corpus
df_plot_summary = pd.read_csv('Data/MovieSummaries/plot_summaries.txt', delimiter='\t', header=None)

In [3]:
# define tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [7]:
# function to clean and tokenize the text
def preprocess_text(text, tokenizer):
    return tokenizer.encode_plus(text, truncation=True, padding='max_length', max_length=tokenizer.model_max_length)

In [8]:
# preprocess the plot summaries
token = []
for i in np.arange(0, len(df_plot_summary[1])):
    token.append(preprocess_text(df_plot_summary[1][i], tokenizer=tokenizer))

In [11]:
print(df_plot_summary[1][0])
print(token[0]['input_ids'])
print(tokenizer.decode(token[0]['input_ids']))

output_string = ''
for ele in tokenizer.convert_ids_to_tokens(token[0]['input_ids']):
    output_string += ele + ' '
print(output_string)

Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.
[101, 14021, 2135, 7724, 1010, 1037, 2524, 1011, 2551, 10095, 4062, 1998, 1048, 7677, 7377, 1010, 1037, 19977, 1010, 4503, 1037, 13576, 2293, 1011, 5223, 3276, 1010, 1998, 2750, 2037, 18024, 2015, 1010, 5382, 2027, 4995, 1005, 1056, 2061, 2367, 2044, 2035, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
# Create batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
# Model evaluation
accuracy = evaluate.load("accuracy")

# ToDo: Revisit this metric
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# ToDo: Specify mappings (id -> label) and (label -> id)

In [ ]:
# define model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2, 
    id2label=id2label, 
    label2id=label2id)

In [13]:
# ToDo: Specify parameters correctly
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

[[7.936819076538086, 2.9820353984832764]]
